# DCGAN 
###  this network is the same architecture as gans but with some enhancements and changes so I followed this architecture 

![DCGAN](images/DCGAN.png)

# import 

In [ ]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D , Conv2DTranspose , Flatten , Reshape  , BatchNormalization , Activation, Flatten ,LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist 
import numpy as np 

# generator 

In [ ]:
class Generator(tf.keras.Model) : 
    def __init__(self  ) : 
        super().__init__() 
        self.dense_1 = Dense(7*7*128 ) 
        self.reshape = Reshape((7,7,128))

    
        self.sequential = Sequential([
             BatchNormalization() ,
             Activation('relu') , 
             Conv2DTranspose(128 , 5 , strides = 2 , padding = 'same') , 
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(64 , 5 , strides = 2  , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(32 , 5   , padding = 'same') ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(1 , 5  , padding = 'same') , 
             Activation('sigmoid')
        ])
    def call(self,  X ) : 
        X = self.reshape(self.dense_1(X))
        X = self.sequential(X)
        return  X 

# Discriminator 

In [ ]:
class Discriminator(tf.keras.Model) : 
    def __init__(self ) : 
        super().__init__() 
        self.sequential = Sequential([
            LeakyReLU(.2), 
            Conv2D(32 , 5 , padding ='same' , strides = 2) , 
            LeakyReLU(.2),
            Conv2D(64 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(128 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(256 , 5 , padding ='same'  ) ,
            Flatten() , 
            Dense(1) , 
            Activation('sigmoid') 
        ])
    def call(self, X) : 
        return self.sequential(X)

# training loop 

In [ ]:
def train(models, x_train, params):

    generator, discriminator, adversarial = models
    batch_size, latent_size, train_steps, model_name = params
    save_interval = 500
    # noise vector to see how the generator output evolves during training
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # number of elements in train dataset
    train_size = x_train.shape[0]
    for i in range(train_steps):

        # select random batch of data 
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]

        ###### train discriminator  
        
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0,
                                  1.0,
                                  size=[batch_size, latent_size])
        # generate fake images
        fake_images = generator.predict(noise)
        # real + fake images = 1 batch of train data
        x = np.concatenate((real_images, fake_images))
        # label real and fake images
        # real images label is 1.0
        y = np.ones([2 * batch_size, 1])
        # fake images label is 0.0
        y[batch_size:, :] = 0.0
        # train discriminator network, log the loss and accuracy
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)



        ######## train the adversarial network for 1 batch
        # 1 batch of fake images with label=1.0
        # since the discriminator weights are frozen in adversarial network
        # only the generator is trained
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0,
                                  1.0, 
                                  size=[batch_size, latent_size])
        # label fake images as real or 1.0
        y = np.ones([batch_size, 1])
        # train the adversarial network 
        # note that unlike in discriminator training, 
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        show=False,
                        step=(i + 1),
                        model_name=model_name)
   
    # save the model after training the generator
    # the trained generator can be reloaded for 
    # future MNIST digit generation
    generator.save(model_name + ".h5")


In [ ]:
def plot_images(generator,
                noise_input,
                show=False,
                step=0,
                model_name="gan"):
    
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

# build the network 

In [ ]:
def build_and_train_models():
    # load MNIST dataset
    (x_train, _), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32') / 255

    model_name = "dcgan_mnist"
    # network parameters
    # the latent or z vector is 100-dim
    latent_size = 100
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 1)

    # build discriminator model
    discriminator = Discriminator()
    optimizer = RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])

    # build generator model
    generator = Generator()

    # build adversarial model
    optimizer = RMSprop(lr=lr * 0.5, decay=decay * 0.5)
    # freeze the weights of discriminator during adversarial training
    discriminator.trainable = False
    # adversarial = generator + discriminator
    adversarial = Sequential([generator,discriminator ])
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])

    # train discriminator and adversarial networks
    models = (generator, discriminator, adversarial)
    params = (batch_size, latent_size, train_steps, model_name)
    train(models, x_train, params)



In [ ]:
build_and_train_models()